# Arnoldi iteration

In [1]:
import numpy as np

#### Krylov basis computation

In [2]:
def compute_krylov_basis(A, b, x0, m):
    """
    Generates a numerical basis for the m-dimensional Krylov subspace.
    
    Parameters:
        A (numpy.ndarray): The matrix.
        b (numpy.ndarray): The vector.
        m (int): The dimensionality of the subspace.
        
    Returns:
        numpy.ndarray: The basis for the Krylov subspace.
    """
    n = A.shape[0]
    
    r0 = b - A @ x0
    
    # Initialize the basis
    basis = np.empty((n, m), dtype = np.float64)
    
    # Set the first column to the initial residual
    basis[:, 0] = r0
    
    # Compute subsequent columns
    for i in range(1, m):
        basis[:, i] = A @ basis[:, i - 1]
        
    return basis

An important measure of the usefulness of a basis is its **conditioning**. It measures how linearly independent vectors are.
If a set of vectors is close to being linearly dependent (or collinear), it means that they are almost providing the same information. Small changes can tip the balance and make one vector a linear combination of the others. This is reflected in a high condition number, indicating that the system is sensitive to changes and hence, numerically unstable.

A condition number close to $1$ means that the basis is well behaved. A condition number close to $10^{16}$ means that the basis is numerically singular (this is related to the fact that standard double precision has around 16 digits of accuracy).

Let us measure the condition number of the Krylov subspace basis as $m$ increases for a $(20,000 \times 20,000)$ matrix.

In [3]:
n = 20000 # Matrix dimension

m = 20

rand = np.random.RandomState(73)
A = rand.rand(n, n)
b = rand.rand(n)
x0 = np.zeros(n)
krylov_basis = compute_krylov_basis(A, b, x0, m)

for m in range(1, m):
    cond_number = np.linalg.cond(krylov_basis[:,:m])
    print(f"{m}: {cond_number:.1e}")

1: 1.0e+00
2: 1.7e+04
3: 1.7e+08
4: 1.7e+12
5: 1.7e+16
6: 4.1e+22
7: 1.1e+27
8: 2.9e+30
9: 1.8e+35
10: 2.9e+37
11: 3.8e+45
12: 2.2e+50
13: 1.2e+56
14: 5.9e+58
15: 8.0e+61
16: 5.8e+66
17: 2.5e+71
18: 7.6e+74
19: 2.2e+78


This is catastrophic. Already with 5 vectors the Krylov subspace basis is effectively numerically singular. **The mathematical reason is that the powers $A^{m}b$ converge to the eigenvector associated with the largest (by magnitude) eigenvalue of $A$**. Hence, the later iterates more and more point in the same direction and are not really linearly dependent. We need a way to generate a stable basis for growing $m$. The ideal case is an orthogonal basis, where all basis vectors are pairwise orthogonal to each other and of unit length.

This is accomplished by the following modified algorithm that after each multiplication with the matrix $A$ orthogonalizes the new vector against all previous vectors.

In [4]:
def arnoldi(A, b, r0, m, tol = 1e-12):
    """
    This function computes an orthonormal basis V_{m+1} = {v_1,...,v_{m+1}} of 
    K_{m+1}(A, r^{(0)}) = span{r^{(0)}, Ar^{(0)}, ..., A^{m}r^{(0)}}.

    Input parameters:
    -----------------
      A: array_like
          An (n x n) array.
      
      b: array_like
          Initial vector of length n.
      
      r0: array_like 
          Initial residual of length n.

      m: int
          One less than the dimension of the Krylov subspace. Must be > 0.
      
      tol: 
          Tolerance for convergence.

    Output:
    -------
      Q: numpy.array 
          n x m array, the columns are an orthonormal basis of the Krylov subspace.
      
      H: numpy.array
          An (m + 1) x m array. It is the matrix A on basis Q. It is upper Hessenberg.
    """
    
    # Check inputs
    n = A.shape[0]
    assert A.shape == (n, n) and b.shape == (n,) and x0.shape == (n,), "Matrix and vector dimensions don not match"
    assert isinstance(m, int) and m > 0, "m must be a positive integer"
    
    m = min(m, n)
    
    # Initialize matrices
    V = np.zeros((n, m + 1))
    H = np.zeros((m + 1, m))
    
    # Normalize input vector and use for Krylov vector
    beta = np.linalg.norm(r0)
    V[:, 0] = r0 / beta

    for k in range(1, m + 1):
        # Generate a new candidate vector
        w = A @ V[:, k - 1] # Note that here is different from arnoldi_one_iter as we iter over k from 1 to m. 
                            # In arnoldi_one_iter we have k as input to the function and here we have V[:, k - 1] as k starts at 1.
        
        # Orthogonalization
        for j in range(k):
            H[j, k - 1] = V[:, j] @ w
            w -= H[j, k - 1] * V[:, j]
        
        H[k, k - 1] = np.linalg.norm(w)

        # Check convergence
        if H[k, k - 1] <= tol:
            print(f"Converged in {k} iterations.")
            return V, H
        
        # Normalize and store the new basis vector
        V[:, k] = w / H[k, k - 1]
    
    return V, H

# Unit tests

### Small matrix just to verify the algorithm is working

In [5]:
n = 10
m = 2
rand = np.random.RandomState(0)

A = rand.rand(n, n)
b = rand.rand(n)
x0 = np.zeros(n)

r0 = b - A @ x0

V, H = arnoldi(A, b, r0, m)
V, H

(array([[ 0.33772937,  0.17493401,  0.45494454],
        [ 0.13453437,  0.62463971, -0.11098119],
        [ 0.36631832,  0.15463533,  0.00101877],
        [ 0.47942078, -0.14437439, -0.49187151],
        [ 0.12394393,  0.28209878,  0.42440875],
        [ 0.28707658,  0.01984779,  0.05898377],
        [ 0.29499125, -0.12367891, -0.09936178],
        [ 0.28513066,  0.11598928,  0.31176799],
        [ 0.11115282,  0.4612879 , -0.49578946],
        [ 0.47471743, -0.46147275,  0.04784273]]),
 array([[3.92980991, 2.03722161],
        [1.98254355, 0.44956505],
        [0.        , 0.52717505]]))

### Check that $V$ is orthogonal

##### Condition number

In [6]:
np.linalg.cond(V)

1.0000000000000002

#### Difference between $V^TV$ and $I$.

In [7]:
np.linalg.norm(V.T @ V - np.eye(m + 1))

6.770054305730027e-16

#### Condition number of the sucessive Krylov bases

In [8]:
for i in range(1, m + 2):
    cond_number = np.linalg.cond(V[:,:i])
    print(f"{cond_number}")

1.0
1.0000000000000002
1.0000000000000002


### Test property: $AV_m = V_{m+1}H_{m+1,m}$

In [9]:
def assert_matrices_equal_with_threshold(matrix1, matrix2, threshold):
    # Check if shapes are equal
    assert len(matrix1) == len(matrix2), "Matrices have different number of rows"
    assert len(matrix1[0]) == len(matrix2[0]), "Matrices have different number of columns"
    
    # Check if all corresponding elements are within the threshold
    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            assert abs(matrix1[i][j] - matrix2[i][j]) <= threshold, f"Element at position ({i}, {j}) is not within the threshold"

assert_matrices_equal_with_threshold(A@V[:,:-1], V@H, 1e-12)

### Trying with a large sparse matrix ($20,000 \times 20,000$) with 1% of nonzero elements (csr format)

In [10]:
from scipy.sparse import random
from scipy.sparse.linalg import eigs

# Generate a large sparse matrix
n = 20000
density = 0.01  # Adjust density as needed
A_sparse = random(n, n, density=density, format='csr')

# Generate a random initial vector and initial approximation
b = np.random.rand(n)
x0 = np.random.rand(n)

# Choose m, the dimension of the Krylov subspace
m = 100  # Adjust m as needed

# Call the arnoldi function
Q, H = arnoldi(A_sparse, b, x0, m)

# Verify orthogonality of Q
orthogonality_error = np.linalg.norm(Q.T @ Q - np.eye(m + 1))
print("Orthogonality error of Q:", orthogonality_error)

# Verify the properties of H
print("Shape of H:", H.shape)

Orthogonality error of Q: 2.0828577121471458e-15
Shape of H: (101, 100)


In [11]:
for i in range(1, m + 2):
    cond_number = np.linalg.cond(Q[:,:i])
    print(f"{cond_number}")

1.0
1.0000000000000002
1.0000000000000009
1.0000000000000007
1.0000000000000004
1.0000000000000009
1.0000000000000007
1.0000000000000007
1.0000000000000009
1.0000000000000009
1.0000000000000004
1.0000000000000002
1.0000000000000009
1.0000000000000009
1.000000000000001
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000009
1.0000000000000004
1.000000000000001
1.0000000000000009
1.0000000000000009
1.0000000000000009
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000007
1.0000000000000009
1.0000000000000007
1.0000000000000009
1.000000000000002
1.000000000000001
1.000000000000001
1.0000000000000018
1.0000000000000022
1.0000000000000013
1.0000000000000013
1.0000000000000018
1.0000000000000016
1.000000000000002
1.000000000000001
1.000000000000001
1.000000000000001
1.000000000000001
1.000000000000002
1.000000000000002
1.000000000000002
1.0000000000000013
1.0000000000000024
1.000000000000001
1.0000000000000027
1.0000000000000024
1.0